# EEG Feature Extraction Pipeline

This notebook extracts multiple types of features from preprocessed EEG data for Alzheimer's disease classification.

## Features Extracted

### 1. Spectral Coherence
- Calculates coherence between all EEG channel pairs
- Measures frequency-domain synchronization
- Output: Channel × Channel × Frequency matrices

### 2. Relative Band Power (RBP)
- Computes power distribution across frequency bands
- Normalizes band power relative to total power
- Bands: Delta (0.5-4Hz), Theta (4-8Hz), Alpha (8-13Hz), Beta (13-30Hz), Gamma (30-100Hz)
- Output: Channel × Band matrices

### 3. Cross-Channel Correlations
- Statistical dependencies between EEG channels
- Time-domain correlation analysis
- Output: Channel × Channel correlation matrices

## Processing Pipeline
1. Load epoched EEG data for each subject
2. Calculate coherence matrices for all channel pairs
3. Compute relative band power for standard frequency bands
4. Save extracted features for machine learning models

## Output Format
- Coherence: `coherence_{subject_id}.npy`
- RBP: `rbp_{subject_id}.npy`
- Features organized by subject for cross-validation

In [ ]:
# Install MNE for EEG processing
!pip install mne

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import mne
import os
from scipy import signal
from scipy.signal import welch

In [ ]:
# Configuration
sampling_rate = 500
base_dir = "/kaggle/input/epochs-of-the-alzheimer-data/epochs_data/"
output_dir = "/kaggle/working/"

# Create output directory
os.makedirs(output_dir, exist_ok=True)

print("Starting spectral coherence calculation for all subjects...")

# Process each subject
for subject_id in range(1, 89):
    subject_file = os.path.join(base_dir, f"subject_{subject_id}.npy")
    
    if os.path.exists(subject_file):
        print(f"Processing subject {subject_id}...")
        
        # Load and transpose EEG data (epochs, channels, samples) -> (epochs, samples, channels)
        eeg_data = np.load(subject_file).transpose(0, 2, 1)
        n_epochs, n_samples, n_channels = eeg_data.shape
        
        # Initialize coherence matrix: (epochs, channels, channels, frequencies)
        coherences = np.zeros((n_epochs, n_channels, n_channels, 129))
        
        # Calculate coherence for each epoch
        for epoch in range(n_epochs):
            for i in range(n_channels):
                for j in range(i + 1, n_channels):
                    # Calculate coherence between channel pairs
                    f, Cxy = signal.coherence(
                        eeg_data[epoch, :, i], 
                        eeg_data[epoch, :, j], 
                        fs=sampling_rate
                    )
                    # Fill symmetric matrix
                    coherences[epoch, i, j] = Cxy
                    coherences[epoch, j, i] = Cxy
            
            if epoch % 10 == 0:  # Progress indicator
                print(f"  Epoch {epoch}/{n_epochs} completed")
        
        # Save coherence data
        coherence_dir = os.path.join(output_dir, "coherence")
        os.makedirs(coherence_dir, exist_ok=True)
        
        output_file = os.path.join(coherence_dir, f"coherence_{subject_id}.npy")
        np.save(output_file, coherences)
        print(f"Subject {subject_id} coherence saved to {output_file}")
    
    else:
        print(f"Subject {subject_id} file not found, skipping.")

print("Coherence calculation completed for all subjects!")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
0
1
2
3
4
5
6
7
8
9
10
1

In [ ]:
def calculate_rbp(signal, fs, window_length, overlap, bands):
    """
    Calculate Relative Band Power (RBP) from EEG signal.
    
    Args:
        signal (np.ndarray): EEG signal with shape (samples, channels)
        fs (float): Sampling frequency in Hz
        window_length (float): Window length in seconds for Welch's method
        overlap (float): Overlap between windows (0 to 1)
        bands (dict): Frequency bands {'band_name': [fmin, fmax]}
    
    Returns:
        np.ndarray: RBP matrix with shape (channels, bands)
    """
    num_channels = signal.shape[1]
    num_bands = len(bands)
    window_samples = int(window_length * fs)
    overlap_samples = int(window_samples * overlap)
    rbp = np.zeros((num_channels, num_bands))

    for channel in range(num_channels):
        channel_signal = signal[:, channel]
        
        # Calculate power spectral density using Welch's method
        f, psd = welch(
            channel_signal, 
            fs, 
            window='hamming', 
            nperseg=window_samples, 
            noverlap=overlap_samples
        )

        # Calculate relative band power for each frequency band
        total_power = np.trapz(psd, f)
        
        for band_idx, (band_name, band_range) in enumerate(bands.items()):
            freq_mask = np.logical_and(f >= band_range[0], f < band_range[1])
            band_power = np.trapz(psd[freq_mask], f[freq_mask])
            rbp[channel, band_idx] = band_power / total_power if total_power > 0 else 0

    return rbp

In [ ]:
# Define frequency bands for RBP calculation
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

# RBP calculation parameters
window_length = 2  # Window length in seconds
overlap = 0.5  # Overlap between windows
fs = 500  # Sampling frequency

print("Frequency bands defined:")
for band_name, (fmin, fmax) in bands.items():
    print(f"  {band_name.capitalize()}: {fmin}-{fmax} Hz")

In [ ]:
print("Starting Relative Band Power (RBP) calculation for all subjects...")

# Setup directories
base_dir = "/kaggle/input/epochs-of-the-alzheimer-data/epochs_data/"
output_dir = "/kaggle/working/"
rbp_dir = os.path.join(output_dir, "rbp")
os.makedirs(rbp_dir, exist_ok=True)

# Process each subject
for subject_id in range(1, 89):
    subject_file = os.path.join(base_dir, f"subject_{subject_id}.npy")
    
    if os.path.exists(subject_file):
        print(f"Processing RBP for subject {subject_id}...")
        
        # Load EEG data and transpose to (epochs, samples, channels)
        eeg_data = np.load(subject_file).transpose(0, 2, 1)
        n_epochs, n_samples, n_channels = eeg_data.shape
        
        # Initialize RBP array: (epochs, channels, bands)
        rbp_values = np.zeros((n_epochs, n_channels, len(bands)))
        
        # Calculate RBP for each epoch
        for epoch in range(n_epochs):
            rbp_values[epoch] = calculate_rbp(
                eeg_data[epoch], 
                fs, 
                window_length, 
                overlap, 
                bands
            )
        
        # Save RBP data
        output_file = os.path.join(rbp_dir, f"rbp_{subject_id}.npy")
        np.save(output_file, rbp_values)
        print(f"Subject {subject_id} RBP saved to {output_file}")
    
    else:
        print(f"Subject {subject_id} file not found, skipping.")

print("RBP calculation completed for all subjects!")

Saved /kaggle/working/rbp/rbp_1.npy
Saved /kaggle/working/rbp/rbp_2.npy
Saved /kaggle/working/rbp/rbp_3.npy
Saved /kaggle/working/rbp/rbp_4.npy
Saved /kaggle/working/rbp/rbp_5.npy
Saved /kaggle/working/rbp/rbp_6.npy
Saved /kaggle/working/rbp/rbp_7.npy
Saved /kaggle/working/rbp/rbp_8.npy
Saved /kaggle/working/rbp/rbp_9.npy
Saved /kaggle/working/rbp/rbp_10.npy
Saved /kaggle/working/rbp/rbp_11.npy
Saved /kaggle/working/rbp/rbp_12.npy
Saved /kaggle/working/rbp/rbp_13.npy
Saved /kaggle/working/rbp/rbp_14.npy
Saved /kaggle/working/rbp/rbp_15.npy
Saved /kaggle/working/rbp/rbp_16.npy
Saved /kaggle/working/rbp/rbp_17.npy
Saved /kaggle/working/rbp/rbp_18.npy
Saved /kaggle/working/rbp/rbp_19.npy
Saved /kaggle/working/rbp/rbp_20.npy
Saved /kaggle/working/rbp/rbp_21.npy
Saved /kaggle/working/rbp/rbp_22.npy
Saved /kaggle/working/rbp/rbp_23.npy
Saved /kaggle/working/rbp/rbp_24.npy
Saved /kaggle/working/rbp/rbp_25.npy
Saved /kaggle/working/rbp/rbp_26.npy
Saved /kaggle/working/rbp/rbp_27.npy
Saved /kag

In [ ]:
def spectral_coherence(x_wavelet, y_wavelet):
    """
    Compute spectral coherence between two wavelet-transformed signals.
    
    Args:
        x_wavelet (np.ndarray): Wavelet coefficients of first signal
        y_wavelet (np.ndarray): Wavelet coefficients of second signal
    
    Returns:
        np.ndarray: Spectral coherence values
    """
    # Calculate power spectral densities
    x_psd = np.mean(np.abs(x_wavelet) ** 2, axis=-1)
    y_psd = np.mean(np.abs(y_wavelet) ** 2, axis=-1)
    
    # Calculate cross power spectral density
    cross_psd = np.mean(x_wavelet * np.conj(y_wavelet), axis=-1)
    
    # Compute coherence
    coherence = np.abs(cross_psd) ** 2 / (x_psd * y_psd + 1e-10)  # Add small epsilon to avoid division by zero
    return coherence

def band_avg(data, band, freqs):
    """
    Average data over specified frequency band.
    
    Args:
        data (np.ndarray): Input data with frequency dimension
        band (tuple): Frequency band limits (fmin, fmax)
        freqs (np.ndarray): Frequency array
    
    Returns:
        np.ndarray: Band-averaged data
    """
    freq_mask = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.mean(data[:, :, freq_mask], axis=2)

def spectral_coherence_connectivity(raw):
    """
    Calculate spectral coherence connectivity across frequency bands.
    
    Args:
        raw (np.ndarray): Raw EEG data with shape (epochs, channels, samples)
    
    Returns:
        np.ndarray: Band-averaged spectral coherence connectivity
    """
    sfreq = 500
    freqs = np.arange(1, 100, 1)  # Frequencies 1-99 Hz
    
    # Apply Morlet wavelet transform
    signals_wavelet = mne.time_frequency.tfr_array_morlet(raw, sfreq, freqs, decim=15)
    n_epochs, n_signals = signals_wavelet.shape[:2]
    
    # Calculate spectral coherence for all channel pairs
    scc = []
    for epoch in range(n_epochs):
        scc_epoch = []
        for i in range(n_signals):   
            for j in range(i + 1, n_signals):
                coherence_vals = spectral_coherence(
                    signals_wavelet[epoch, i], 
                    signals_wavelet[epoch, j]
                )
                scc_epoch.append(coherence_vals)
        scc.append(scc_epoch)
    
    # Define frequency bands
    frequency_bands = {
        'delta': (0.5, 4),
        'theta': (4, 8), 
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 100)
    }
    
    # Convert to array and average over frequency bands
    data = np.array(scc)
    freqs_range = np.linspace(0.5, 100, 99)
    
    # Calculate band averages
    band_averages = []
    for band_name, band_range in frequency_bands.items():
        band_avg_data = band_avg(data, band_range, freqs_range)
        band_averages.append(band_avg_data)
    
    # Combine band-averaged data: (epochs, connections, bands)
    band_data = np.stack(band_averages, axis=2)
    
    return band_data

In [ ]:
print("Starting Spectral Coherence Connectivity (SCC) calculation for all subjects...")

# Setup output directory
output_dir = "/kaggle/working/scc_cleaned_base/"
os.makedirs(output_dir, exist_ok=True)

# Process each subject
for subject in range(1, 89):
    file_path = f"/kaggle/input/epochs-of-the-alzheimer-data/epochs_data/subject_{subject}.npy"
    output_file = os.path.join(output_dir, f"sub-{subject}_epochs.npy")
    
    if os.path.exists(file_path):
        print(f"Processing SCC for subject {subject}...")
        
        # Load EEG data
        data = np.load(file_path)
        
        # Calculate spectral coherence connectivity
        scc_data = spectral_coherence_connectivity(data)
        
        # Save SCC data
        np.save(output_file, scc_data)
        print(f"Subject {subject} SCC saved to {output_file}")
        
    else:
        print(f"Subject {subject} file not found, skipping.")

print("SCC calculation completed for all subjects!")

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   25.3s


Subject 1: Spectral Coherence Connectivity saved to /kaggle/working/scc_cleaned_base/subject_1_scc.npy


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   31.6s


Subject 2: Spectral Coherence Connectivity saved to /kaggle/working/scc_cleaned_base/subject_2_scc.npy


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   11.7s


...

Subject 86: Spectral Coherence Connectivity saved to /kaggle/working/scc_cleaned_base/subject_86_scc.npy


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   24.2s


Subject 87: Spectral Coherence Connectivity saved to /kaggle/working/scc_cleaned_base/subject_87_scc.npy


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   31.5s


Subject 88: Spectral Coherence Connectivity saved to /kaggle/working/scc_cleaned_base/subject_88_scc.npy
